# Fine-tuning  d'un réseau à l'aide de l'API Keras

L'un des plus grands avantages de l'apprentissage par transfert est peut-être sa capacité à saisir le vent arrière produit par les connaissances encodées dans des réseaux pré-formés. En échangeant simplement les couches les moins profondes dans l'un de ces réseaux, nous pouvons obtenir des performances remarquables sur de nouveaux ensembles de données non liés, même si nos données sont petites. Pourquoi? Parce que les informations contenues dans les couches inférieures sont pratiquement universelles : elles codent des formes et des formes de base qui s'appliquent à presque tous les problèmes de vision par ordinateur.

Dans cette recette, nous allons affiner un réseau VGG16 pré-entraîné sur un petit ensemble de données, en obtenant un score de haute précision par ailleurs improbable

Nous utiliserons un ensemble de données appelé 17 Category Flower Dataset, disponible ici : http://www.robots.ox.ac.uk/~vgg/data/flowers/17. Une version organisée en sous-dossiers par classe est disponible ici : https://github.com/PacktPublishing/Tensorflow-2.0-Computer-Vision-Cookbook/tree/master/ch3/recipe3/flowers17.zip. Téléchargez-le et décompressez-le à l'emplacement de votre choix.

In [3]:
!cp /content/drive/MyDrive/probabilité/flowers17.zip /content/
!unzip flowers17.zip

Archive:  flowers17.zip
   creating: flowers17/
   creating: flowers17/images/
   creating: flowers17/images/bluebell/
  inflating: flowers17/images/bluebell/image_0241.jpg  
  inflating: flowers17/images/bluebell/image_0242.jpg  
  inflating: flowers17/images/bluebell/image_0243.jpg  
  inflating: flowers17/images/bluebell/image_0244.jpg  
  inflating: flowers17/images/bluebell/image_0245.jpg  
  inflating: flowers17/images/bluebell/image_0246.jpg  
  inflating: flowers17/images/bluebell/image_0247.jpg  
  inflating: flowers17/images/bluebell/image_0248.jpg  
  inflating: flowers17/images/bluebell/image_0249.jpg  
  inflating: flowers17/images/bluebell/image_0250.jpg  
  inflating: flowers17/images/bluebell/image_0251.jpg  
  inflating: flowers17/images/bluebell/image_0252.jpg  
  inflating: flowers17/images/bluebell/image_0253.jpg  
  inflating: flowers17/images/bluebell/image_0254.jpg  
  inflating: flowers17/images/bluebell/image_0255.jpg  
  inflating: flowers17/images/bluebell/im

Le fine tuning est facile ! Suivez ces étapes pour terminer cette recette

**1.** Importez les dépendances nécessaires :

In [4]:
import os
import pathlib
from glob import glob

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import *

**2.** Définissez la graine aléatoire :

In [5]:
SEED = 999

**3.** Défnir une fonction qui construira un nouveau réseau à partir d'un modèle pré-entraîné, où les couches supérieures entièrement connectées seront toutes nouvelles et adaptées au problème à résoudre

In [6]:
def build_network(base_model, classes):
    x = Flatten()(base_model.output)
    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return output

**4.** Définir une fonction qui chargera les images et les étiquettes dans l'ensemble de données en tant que tableaux NumPy

In [7]:
def load_images_and_labels(image_paths,
                           target_size=(256, 256)):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

**5.** Chargez les chemins des images et extrayez-en l'ensemble des classes :

In [8]:
dataset_path = pathlib.Path('/content/flowers17') 
files_pattern = (dataset_path / 'images' / '*' / '*.jpg')
image_paths = [*glob(str(files_pattern))]
CLASSES = {p.split(os.path.sep)[-2] for p in image_paths}

**6.** Chargez les images et normalisez-les, encodez à one-hot les étiquettes avec LabelBinarizer() et divisez les données en sous-ensembles pour l'entraînement (80%) et les tests (20%) :

In [9]:
X, y = load_images_and_labels(image_paths)
X = X.astype('float') / 255.0
y = LabelBinarizer().fit_transform(y)
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     random_state=SEED)

**7.** Instanciez un VGG16 pré-entraîné, sans les couches supérieures. Spécifiez une forme d'entrée de 256x256x3

In [10]:
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_tensor=Input(shape=(256, 256, 3)))

58900480/58889256 [==============================] - 1s 0us/step


In [11]:
#Gelez toutes les couches du modèle de base. Nous le faisons parce que nous ne voulons pas les recycler, mais utiliser leurs connaissances existantes
for layer in base_model.layers:
    layer.trainable = False

**8.** Construisez le réseau complet avec un nouvel ensemble de couches au-dessus en utilisant build_network() (défini à l'étape 3)

In [12]:
model = build_network(base_model, len(CLASSES))
model = Model(base_model.input, model)

**9.** Déﬁnissez la taille du lot et un ensemble d'augmentations à appliquer via ImageDataGenerator() :

In [13]:
BATCH_SIZE = 64
augmenter = ImageDataGenerator(rotation_range=30,
                               horizontal_flip=True,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')
train_generator = augmenter.flow(X_train, y_train, BATCH_SIZE)

**10.** Réchauffez le réseau. Cela signifie que nous n'entraînerons les nouvelles couches (les autres sont gelées) que pendant 20 époques, en utilisant RMSProp avec un taux d'apprentissage de 0,001. Enfin, nous évaluerons le réseau sur l'ensemble de test :

In [14]:
WARMING_EPOCHS = 20
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-3),
              metrics=['accuracy'])
history = model.fit(train_generator,
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    epochs=WARMING_EPOCHS)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
17/17 [==============================] - 83s 2s/step - loss: 1.5857 - accuracy: 0.5340 - val_loss: 2.9972 - val_accuracy: 0.5221
Epoch 2/20
17/17 [==============================] - 21s 1s/step - loss: 0.7540 - accuracy: 0.7812 - val_loss: 2.3457 - val_accuracy: 0.5699
Epoch 3/20
17/17 [==============================] - 21s 1s/step - loss: 0.5446 - accuracy: 0.8631 - val_loss: 2.0590 - val_accuracy: 0.6029
Epoch 4/20
17/17 [==============================] - 21s 1s/step - loss: 0.4343 - accuracy: 0.8980 - val_loss: 1.3152 - val_accuracy: 0.6765
Epoch 5/20
17/17 [==============================] - 21s 1s/step - loss: 0.3570 - accuracy: 0.9154 - val_loss: 1.1266 - val_accuracy: 0.7096
Epoch 6/20
17/17 [==============================] - 21s 1s/step - loss: 0.3225 - accuracy: 0.9035 - val_loss: 1.1564 - val_accuracy: 0.7169
Epoch 7/20
17/17 [==============================] - 21s 1s/step - loss: 0.3018 - accuracy: 0.9256 - val_loss: 0.8495 - val_accuracy: 0.8051
Epoch 8/20
17/17 [==

In [15]:
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

9/9 [==============================] - 3s 275ms/step - loss: 0.7199 - accuracy: 0.7978
Test accuracy: 0.7977941036224365


**11.** Maintenant que le réseau a été réchauffé, nous allons affiner les couches finales du modèle de base, en particulier à partir du 16 (rappelez-vous, l'indexation zéro), ainsi que les couches entièrement connectées, pour 50 époques, en utilisant SGD avec un taux d'apprentissage de 0,001 :

In [16]:
EPOCHS = 50
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-3),
              metrics=['accuracy'])
history = model.fit(train_generator,
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
17/17 [==============================] - 23s 1s/step - loss: 0.1083 - accuracy: 0.9825 - val_loss: 0.5124 - val_accuracy: 0.8640
Epoch 2/50
17/17 [==============================] - 21s 1s/step - loss: 0.1162 - accuracy: 0.9807 - val_loss: 0.4504 - val_accuracy: 0.8750
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 0.1220 - accuracy: 0.9752 - val_loss: 0.4155 - val_accuracy: 0.8824
Epoch 4/50
17/17 [==============================] - 21s 1s/step - loss: 0.0990 - accuracy: 0.9789 - val_loss: 0.3919 - val_accuracy: 0.8897
Epoch 5/50
17/17 [==============================] - 21s 1s/step - loss: 0.1106 - accuracy: 0.9807 - val_loss: 0.3759 - val_accuracy: 0.8860
Epoch 6/50
17/17 [==============================] - 21s 1s/step - loss: 0.0947 - accuracy: 0.9835 - val_loss: 0.3646 - val_accuracy: 0.8860
Epoch 7/50
17/17 [==============================] - 21s 1s/step - loss: 0.1054 - accuracy: 0.9853 - val_loss: 0.3570 - val_accuracy: 0.8897
Epoch 8/50
17/17 [==

Après le préchauffage, le réseau a atteint une précision de 79,77 % sur l'ensemble de test. Ensuite, lorsque nous l'avons affiné, après 50 époques, la précision est passée à 91,17% sur l'ensemble de test. Nous verrons comment tout cela fonctionne dans la section suivante.

Nous avons réussi à exploiter les connaissances d'un VGG16 pré-entraîné sur l'énorme base de données ImageNet. En remplaçant les couches supérieures, entièrement connectées et chargées de la classification proprement dite (les autres faisant office d'extracteurs de caractéristiques), par notre propre ensemble de couches profondes adaptées à notre problème, nous avons réussi à obtenir un 91,7 % de précision sur l'ensemble de test.

Ce résultat est une démonstration de la puissance de l'apprentissage par transfert, d'autant plus que nous n'avons que 81 images par classe dans l'ensemble de données (81x17 = 1 377 au total), une quantité insuffisante pour former un modèle d'apprentissage en profondeur performant à partir de zéro.

Vous pouvez en savoir plus sur les modèles pré-entraînés Keras ici : https://www.tensorflow.org/api_docs/python/tf/keras/applications